<a href="https://colab.research.google.com/github/prathamchintamani/neuralnets/blob/main/bank_note_authentication_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://archive.ics.uci.edu/dataset/267/banknote+authentication
dataset

In [113]:
#libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [114]:
import torch
from torch import nn

In [115]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

In [116]:
#data loading
df = pd.read_csv("/content/data_banknote_authentication.txt")
df.columns = ["variance","skewness","curtosis","entropy","target"]
df = df.sample(frac = 1)
df = df.reset_index(drop = True)


In [117]:
#preprocessing
y = df.target
X = df.drop(["target"],axis = 1)
X_train ,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.70,random_state = 0)

In [118]:
#pddf to numpy
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [119]:
#assigning number of rows to variables
n = np.shape(X_train)[0]
m = np.shape(X_test)[0]

In [120]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [121]:
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)

In [122]:
class neural(nn.Module):
  def __init__(self, in_size, hidden_size = 8):
    super().__init__()
    self.layer1 = nn.Linear(in_features=in_size,out_features=hidden_size)
    self.layer2 = nn.Linear(in_features=hidden_size,out_features=1)
    self.relu=nn.ReLU()
    self.sigmoid=nn.Sigmoid()
  def forward(self, X):
    return self.sigmoid(self.layer2(self.relu(self.layer1(X))))


In [123]:
model = neural(in_size = 4, hidden_size = 10).to(device)

In [124]:
print(model)

neural(
  (layer1): Linear(in_features=4, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [125]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [126]:
torch.manual_seed(42)
epochs = 1000

In [127]:
for epoch in range(epochs):
    # 1. Forward pass
    y_logits = model(X_train).squeeze()
    y_pred = torch.round(y_logits) # logits -> prediction probabilities -> prediction labels

    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model(X_test).squeeze()
      test_pred = torch.round(test_logits) # logits -> prediction probabilities -> prediction labels
      # 2. Calcuate loss and accuracy
      test_loss = loss_fn(test_logits, y_test)
      test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.80373, Accuracy: 35.14% | Test Loss: 0.77886, Test Accuracy: 39.56%
Epoch: 100 | Loss: 0.59736, Accuracy: 89.26% | Test Loss: 0.58864, Test Accuracy: 89.08%
Epoch: 200 | Loss: 0.56190, Accuracy: 97.81% | Test Loss: 0.55576, Test Accuracy: 96.60%
Epoch: 300 | Loss: 0.55102, Accuracy: 98.96% | Test Loss: 0.54489, Test Accuracy: 98.54%
Epoch: 400 | Loss: 0.54520, Accuracy: 99.27% | Test Loss: 0.53899, Test Accuracy: 99.27%
Epoch: 500 | Loss: 0.54149, Accuracy: 99.79% | Test Loss: 0.53522, Test Accuracy: 99.27%
Epoch: 600 | Loss: 0.53893, Accuracy: 99.90% | Test Loss: 0.53261, Test Accuracy: 99.76%
Epoch: 700 | Loss: 0.53707, Accuracy: 99.90% | Test Loss: 0.53070, Test Accuracy: 99.76%
Epoch: 800 | Loss: 0.53567, Accuracy: 99.90% | Test Loss: 0.52925, Test Accuracy: 99.76%
Epoch: 900 | Loss: 0.53458, Accuracy: 100.00% | Test Loss: 0.52811, Test Accuracy: 99.76%
